<a href="https://colab.research.google.com/github/HazelvdW/context-framed-listening/blob/main/NLP_framed_listening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP analysis for Framed Listening study.
> Authored by **Hazel A. van der Walle** (PhD student, Music, Durham University), September 2025.

All datasets generated and used for this study are openly available on GitHub https://github.com/HazelvdW/context-framed-listening.

The cleaned raw data (processed in R) are used in this notebook, so let's clone necessary files and directories:

In [1]:
!git clone https://github.com/HazelvdW/context-framed-listening.git

Cloning into 'context-framed-listening'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 17 (delta 5), reused 9 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 222.37 KiB | 940.00 KiB/s, done.
Resolving deltas: 100% (5/5), done.


You should now have a file called **"context-framed-listening"** in this notebook.

< Check this out by clicking on the folder icon on the lefthand side panel in this webpage.

For this NLP analysis, we are only working from the file **"data_study1_MAIN.csv"** which contains participants qualitative thought descriptions.





---
## Setup

Start by importing the necessary packages in the cell below:

In [2]:
# importing standard Python libraries
import os
import csv
import pandas as pd
import numpy as np

Load in the data .csv file:

In [4]:
data = pd.read_csv("/content/context-framed-listening/data_study1_MAIN.csv")

This dataset contains every participant's response to all 16 clip-context stimuli pairings.

_For the purposes of this analysis_ we are only interested in trials where music-evoked thoughts (METs) were expereienced and described – this is all rows where "descr_THOUGHT.text" is _not_ NA.

Let's create a new dataset that only contains trials with METs:

In [10]:
dataMET = data[data['descr_THOUGHT.text'].notna()].copy()

Familiarise yourself with the data structure by taking a quick look through before we dig into any analyses.

In [11]:
display(dataMET)

# print out all column headers so we have a quick copy for later reference
print(dataMET.columns)

,clip_name,context_word,expName,PROLIFIC_PID,File_ID,date,response_thought_or_not.keys,descr_THOUGHT.text,rating_music_prompted.response,rating_spontaneity.response,...,demographics.livingCountry,demographics.birthCountry,demographics.nativeLanguage,demographics.otherLanguage,demographics.otherLanguageText,demographics.hearingImpariments,demographics.hearingImpairmentsText,demographics.education,demographics.musicianIdentification,demographics.feedback
0,80s_LOW_02_Breaking_Away.mp3,bar,clip_context_g1,5eff5f05b92981000a2aed73,clip_context_g1_5eff5f05b92981000a2aed73_02059...,2025-07-01_10h45.13.126,y,"kind of sad, melancholy. not happy or upbeat. ...",5.0,4.0,...,United Kingdom,United Kingdom,English,False,NaN,False,NaN,5,2,NaN
1,Jazz_MED_07_Turiya_and_Ramakrishna.mp3,video game,clip_context_g1,5eff5f05b92981000a2aed73,clip_context_g1_5eff5f05b92981000a2aed73_02059...,2025-07-01_10h45.13.126,y,it did not sound like a video game. if anythin...,5.0,5.0,...,United Kingdom,United Kingdom,English,False,NaN,False,NaN,5,2,NaN
2,80s_MED_08_After_Tonight.mp3,video game,clip_context_g1,5eff5f05b92981000a2aed73,clip_context_g1_5eff5f05b92981000a2aed73_02059...,2025-07-01_10h45.13.126,y,"overly upbeat. no real emotions, peppy. too mu...",5.0,4.0,...,United Kingdom,United Kingdom,English,False,NaN,False,NaN,5,2,NaN
3,Metal_LOW_09_Darkside.mp3,concert,clip_context_g1,5eff5f05b92981000a2aed73,clip_context_g1_5eff5f05b92981000a2aed73_02059...,2025-07-01_10h45.13.126,y,"very heavy rock, not for me. somewhere that i ...",5.0,5.0,...,United Kingdom,United Kingdom,English,False,NaN,False,NaN,5,2,NaN
4,Metal_MED_20_Welcome_to_the_Family.mp3,video game,clip_context_g1,5eff5f05b92981000a2aed73,clip_context_g1_5eff5f05b92981000a2aed73_02059...,2025-07-01_10h45.13.126,y,"very charged, maybe you've won something or wo...",5.0,4.0,...,United Kingdom,United Kingdom,English,False,NaN,False,NaN,5,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2554,Metal_MED_20_Welcome_to_the_Family.mp3,concert,clip_context_g4,6824fc226d9b4777f8695cf0,clip_context_g4_PROLIFIC_PID_992291.csv,2025-07-01_06h12.29.151,y,A rock band made up of teenage white kids play...,5.0,5.0,...,United States,United States,English,False,NaN,False,NaN,3,2,none
2555,80s_LOW_02_Breaking_Away.mp3,concert,clip_context_g4,6824fc226d9b4777f8695cf0,clip_context_g4_PROLIFIC_PID_992291.csv,2025-07-01_06h12.29.151,y,People in a ballroom in elegant dresses slow d...,5.0,5.0,...,United States,United States,English,False,NaN,False,NaN,3,2,none
2557,Jazz_MED_02_I_Guess_Ill_Hang_My_Tears_Out_To_D...,video game,clip_context_g4,6824fc226d9b4777f8695cf0,clip_context_g4_PROLIFIC_PID_992291.csv,2025-07-01_06h12.29.151,y,I imagined a jazz festival and old men on stag...,5.0,5.0,...,United States,United States,English,False,NaN,False,NaN,3,2,none
2558,Electronic_MED_20_The_Distance.mp3,movie,clip_context_g4,6824fc226d9b4777f8695cf0,clip_context_g4_PROLIFIC_PID_992291.csv,2025-07-01_06h12.29.151,y,I imagined a documentary mostly about fun fact...,5.0,5.0,...,United States,United States,English,False,NaN,False,NaN,3,2,none


Index(['clip_name', 'context_word', 'expName', 'PROLIFIC_PID', 'File_ID',
       'date', 'response_thought_or_not.keys', 'descr_THOUGHT.text',
       'rating_music_prompted.response', 'rating_spontaneity.response',
       'rating_novelty.response', 'input_NOT.text',
       'rating_familiarity.response', 'rating_enjoyment.response',
       'demographics.headphones', 'demographics.age', 'demographics.gender',
       'demographics.livingCountry', 'demographics.birthCountry',
       'demographics.nativeLanguage', 'demographics.otherLanguage',
       'demographics.otherLanguageText', 'demographics.hearingImpariments',
       'demographics.hearingImpairmentsText', 'demographics.education',
       'demographics.musicianIdentification', 'demographics.feedback'],
      dtype='object')


We're going to combine the clip and context columns to make an additional indicator that we can use in functions later.

We can also drop the columns that where only relevant to distinguishing have- and have-not thoughts.

In [19]:
def create_clip_context_pair(row):
    if 'bar' in row['context_word']:
        return 'BAR-' + row['clip_name']
    elif 'video game' in row['context_word']:
        return 'VIDEOGAME-' + row['clip_name']
    elif 'concert' in row['context_word']:
        return 'CONCERT-' + row['clip_name']
    elif 'movie' in row['context_word']:
        return 'MOVIE-' + row['clip_name']
    else:
        return row['clip_name'] # Default to just the clip name if no match

dataMET['clip_context_PAIR'] = dataMET.apply(create_clip_context_pair, axis=1)

dataMET.drop(columns = ['response_thought_or_not.keys', 'input_NOT.text'],
             inplace=True)

# Check the dataframe by a quick re-view :)
display(dataMET)
print(dataMET.columns)

dataMET.to_csv('/content/context-framed-listening/dataMET.csv', encoding='utf-8')

---
## Descriptive Statistics

Below we are going to run some basic descriptive statistics on the clip-context pairings.


We're going to create a dataframe that includes summary info about each clip-context stimuli pairing including:

* Number of participants that reported METs while listening
* Mean ratings of MET "prompting-power", spontanteity, and novelty, and clip familiarity and enjoyment

In [21]:
columns = dataMET.columns.tolist()[1:-1]

# Drop these following columns so they don't aggregate by clip-context grouping:
drop = ['clip_name', 'context_word', 'expName', 'File_ID', 'date',
        'descr_THOUGHT.text', 'demographics.headphones', 'demographics.age',
        'demographics.gender','demographics.livingCountry',
        'demographics.birthCountry', 'demographics.nativeLanguage',
        'demographics.otherLanguage', 'demographics.otherLanguageText',
        'demographics.hearingImpariments', 'demographics.hearingImpairmentsText',
        'demographics.education','demographics.musicianIdentification',
        'demographics.feedback']

# Setting up an aggregate function collector
agg_fun = {}

# As we dropped trials without METs, we can just sum participants for MET occurrence
agg_fun['PROLIFIC_PID'] = 'count'

# Taking the mean of all columns except participant IDs and dropped columns
for col in columns:
    if col not in drop and col != 'PROLIFIC_PID':
        agg_fun[col] = 'mean'

# Group the dataframe by clip-context pairing, then run the aggregate functions created above
clipContextDescrStats = dataMET.groupby('clip_context_PAIR').agg(agg_fun)
display(clipContextDescrStats)

clipContextDescrStats.to_csv('/content/context-framed-listening/clipContextDescrStats.csv', encoding='utf-8')

,PROLIFIC_PID,rating_music_prompted.response,rating_spontaneity.response,rating_novelty.response,rating_familiarity.response,rating_enjoyment.response
clip_context_PAIR,,,,,,
BAR-80s_LOW_02_Breaking_Away.mp3,34,4.235294,3.882353,3.382353,2.294118,3.705882
BAR-80s_LOW_06_Summer_Beach.mp3,29,4.448276,4.103448,3.482759,2.586207,3.793103
BAR-80s_MED_08_After_Tonight.mp3,29,4.275862,3.862069,2.965517,2.000000,3.310345
BAR-80s_MED_13_Your_Love_Drives_Me_Crazy.mp3,25,4.160000,3.840000,2.360000,2.280000,3.600000
BAR-Electronic_LOW_09_Expansion.mp3,27,4.592593,3.962963,2.259259,2.000000,3.259259
...,...,...,...,...,...,...
VIDEOGAME-Jazz_MED_07_Turiya_and_Ramakrishna.mp3,35,4.428571,4.285714,2.685714,2.457143,3.742857
VIDEOGAME-Metal_LOW_09_Darkside.mp3,34,4.558824,3.794118,2.411765,2.029412,3.235294
VIDEOGAME-Metal_LOW_14_Viaje_Por_Existir.mp3,26,4.307692,3.923077,2.423077,2.230769,3.346154


We can check here what the minimum and maximum reported MET occurences were to each clip-context stimuli pairing.

In [24]:
mostMETs_ccpair = clipContextDescrStats['PROLIFIC_PID'].idxmax()
leastMETs_ccpair = clipContextDescrStats['PROLIFIC_PID'].idxmin()
mostMETs_value = clipContextDescrStats['PROLIFIC_PID'].max()
leastMETs_value = clipContextDescrStats['PROLIFIC_PID'].min()

print(f"Clip-context pair with the most reported METs: {mostMETs_ccpair} ({mostMETs_value})")
print(f"Clip-context pair with the least reported METs: {leastMETs_ccpair} ({leastMETs_value})")

Clip-context pair with the most reported METs: BAR-Electronic_LOW_14_Tape.mp3 (35)
Clip-context pair with the least reported METs: BAR-80s_MED_13_Your_Love_Drives_Me_Crazy.mp3 (25)




---



##

In [ ]:
import re

def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        return text.lower()
    return text

dataMET['cleaned_MET_descr'] = dataMET['MET_descr'].apply(clean_text)
display(dataMET[['MET_descr', 'cleaned_MET_descr']].head())

/tmp/ipython-input-1650317474.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataMET['cleaned_MET_descr'] = dataMET['MET_descr'].apply(clean_text)


,MET_descr,cleaned_MET_descr
0,"kind of sad, melancholy. not happy or upbeat. ...",kind of sad melancholy not happy or upbeat emo...
1,it did not sound like a video game. if anythin...,it did not sound like a video game if anything...
2,"overly upbeat. no real emotions, peppy. too mu...",overly upbeat no real emotions peppy too much
3,"very heavy rock, not for me. somewhere that i ...",very heavy rock not for me somewhere that i do...
4,"very charged, maybe you've won something or wo...",very charged maybe youve won something or won ...
